In [1]:
import os
import numpy as np
import pretty_midi
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings

2023-08-14 00:39:22.243094: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Ignore specific warning from pretty_midi
warnings.filterwarnings('ignore', category=RuntimeWarning, module='pretty_midi.pretty_midi')

In [3]:
# Function to convert MIDI to piano roll
def midi_to_piano_roll(midi_file):
    try:
        midi = pretty_midi.PrettyMIDI(midi_file)
        piano_roll = midi.get_piano_roll(fs=25)
        return np.asarray(piano_roll)
    except Exception as e:
        print(f"Error in {midi_file}: {e}")
        return None

# Load the data
def load_data_from_folder(folder):
    composers = os.listdir(folder)
    labels, piano_rolls = [], []
    for composer in composers:
        composer_path = os.path.join(folder, composer)
        if os.path.isdir(composer_path):
            for midi_file in os.listdir(composer_path):
                if midi_file.endswith('.midi') or midi_file.endswith('.mid'):
                    piano_roll = midi_to_piano_roll(os.path.join(composer_path, midi_file))
                    if piano_roll is not None:
                        labels.append(composer)
                        piano_rolls.append(piano_roll)
    return piano_rolls, labels

In [4]:
train_data, train_labels = load_data_from_folder('/Users/fedijay/Desktop/USD/AAI_511_02_SUM23_NeuralNetworks_and_DeepLearning/FP_AAI_511_NNL_SUM2_Team3/Composer_Dataset2/NN_midi_files_extended/train')
dev_data, dev_labels = load_data_from_folder('/Users/fedijay/Desktop/USD/AAI_511_02_SUM23_NeuralNetworks_and_DeepLearning/FP_AAI_511_NNL_SUM2_Team3/Composer_Dataset2/NN_midi_files_extended/dev')
test_data, test_labels = load_data_from_folder('/Users/fedijay/Desktop/USD/AAI_511_02_SUM23_NeuralNetworks_and_DeepLearning/FP_AAI_511_NNL_SUM2_Team3/Composer_Dataset2/NN_midi_files_extended/test')

In [5]:
# Encoding labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_labels)
y_train = to_categorical(y_train)

y_dev = label_encoder.transform(dev_labels)
y_dev = to_categorical(y_dev)

y_test = label_encoder.transform(test_labels)
y_test = to_categorical(y_test)

In [6]:
sample_shape = train_data[0].shape
print(sample_shape)

(128, 14016)


In [7]:
sample_shape = dev_data[0].shape
print(sample_shape)

(128, 8764)


In [8]:
sample_shape = test_data[0].shape
print(sample_shape)

(128, 10580)


In [9]:
fixed_length = 10627  # Chosen based on the 75th percentile

# Function to limit sequences to a fixed length
def limit_sequence_length(data, fixed_length):
    limited_data = []
    for sample in data:
        if sample.shape[1] > fixed_length:
            limited_sample = sample[:, :fixed_length]
        else:
            limited_sample = np.pad(sample, ((0, 0), (0, fixed_length - sample.shape[1])), mode='constant', constant_values=0)
        limited_data.append(limited_sample)
    return np.array(limited_data)

# Adjust all datasets
X_train = limit_sequence_length(train_data, fixed_length).reshape(-1, 128, fixed_length, 1)
X_dev = limit_sequence_length(dev_data, fixed_length).reshape(-1, 128, fixed_length, 1)
X_test = limit_sequence_length(test_data, fixed_length).reshape(-1, 128, fixed_length, 1)

In [10]:
number_of_classes = y_train.shape[1]

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 10627, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(number_of_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)

Epoch 1/10
12/12 [==============================] - 240s 20s/step - loss: 1494.0325 - accuracy: 0.1978 - val_loss: 158.5065 - val_accuracy: 0.2857
Epoch 2/10
12/12 [==============================] - 234s 20s/step - loss: 48.1691 - accuracy: 0.4011 - val_loss: 16.9934 - val_accuracy: 0.2571
Epoch 3/10
12/12 [==============================] - 234s 20s/step - loss: 6.2980 - accuracy: 0.5420 - val_loss: 6.8858 - val_accuracy: 0.2857
Epoch 4/10
12/12 [==============================] - 234s 19s/step - loss: 1.0899 - accuracy: 0.7561 - val_loss: 2.8727 - val_accuracy: 0.2857
Epoch 5/10
12/12 [==============================] - 235s 20s/step - loss: 0.3329 - accuracy: 0.9241 - val_loss: 3.3551 - val_accuracy: 0.3143
Epoch 6/10
12/12 [==============================] - 234s 19s/step - loss: 0.1852 - accuracy: 0.9458 - val_loss: 3.3819 - val_accuracy: 0.2857
Epoch 7/10
12/12 [==============================] - 234s 20s/step - loss: 0.0936 - accuracy: 0.9729 - val_loss: 3.6198 - val_accuracy: 0.2857

In [11]:
import pandas as pd
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [12]:
# Load CSV data
train_df = pd.read_csv('/Users/fedijay/Desktop/USD/AAI_511_02_SUM23_NeuralNetworks_and_DeepLearning/FP_AAI_511_NNL_SUM2_Team3/csv_file/midi_train_processed.csv')
dev_df = pd.read_csv('/Users/fedijay/Desktop/USD/AAI_511_02_SUM23_NeuralNetworks_and_DeepLearning/FP_AAI_511_NNL_SUM2_Team3/csv_file/midi_val_processed.csv')
test_df = pd.read_csv('/Users/fedijay/Desktop/USD/AAI_511_02_SUM23_NeuralNetworks_and_DeepLearning/FP_AAI_511_NNL_SUM2_Team3/csv_file/midi_test_processed.csv')

In [13]:
timesteps = 128
features_per_timestep = X_train.shape[1] // timesteps

In [14]:
num_sequences = X_train.shape[1] // timesteps
print(num_sequences)

1


In [15]:
features_per_timestep = X_train.shape[1] // num_sequences
print(features_per_timestep)

128


In [16]:
# Extract features and labels
X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values
X_dev = dev_df.iloc[:, :-1].values
y_dev = dev_df.iloc[:, -1].values
X_test = test_df.iloc[:, :-1].values
y_test = test_df.iloc[:, -1].values

# Define fixed values based on the provided info
timesteps = 128
num_sequences = 117
features_per_timestep = 128

# Pad or truncate sequences to a fixed length
def pad_or_truncate(data, fixed_length):
    padded_data = []
    for sample in data:
        if len(sample) < fixed_length:
            padded_data.append(np.pad(sample, (0, fixed_length - len(sample)), 'constant'))
        else:
            padded_data.append(sample[:fixed_length])
    return np.array(padded_data)

# Adjusting the X_train, X_dev and X_test to match the expected length
expected_length = timesteps * features_per_timestep
X_train = pad_or_truncate(X_train, expected_length)
X_dev = pad_or_truncate(X_dev, expected_length)
X_test = pad_or_truncate(X_test, expected_length)

# Reshape data for CNN
X_train = X_train.reshape(-1, timesteps, features_per_timestep, 1)
X_dev = X_dev.reshape(-1, timesteps, features_per_timestep, 1)
X_test = X_test.reshape(-1, timesteps, features_per_timestep, 1)

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_dev_encoded = label_encoder.transform(y_dev)
y_test_encoded = label_encoder.transform(y_test)

y_train_categorical = to_categorical(y_train_encoded)
y_dev_categorical = to_categorical(y_dev_encoded)
y_test_categorical = to_categorical(y_test_encoded)

In [17]:
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Dropout, BatchNormalization
from keras.callbacks import EarlyStopping

model = Sequential()

# First Convolutional Layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second Convolutional Layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and Fully Connected Layers-
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

# Use RMSprop optimizer as an alternative
optimizer = RMSprop(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train_categorical, validation_data=(X_dev, y_dev_categorical), epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
12/12 [==============================] - 4s 285ms/step - loss: 12.1632 - accuracy: 0.0941 - val_loss: 22.6470 - val_accuracy: 0.0857
Epoch 2/100
12/12 [==============================] - 3s 253ms/step - loss: 2.1849 - accuracy: 0.2285 - val_loss: 23.1417 - val_accuracy: 0.0857
Epoch 3/100
12/12 [==============================] - 3s 253ms/step - loss: 1.7942 - accuracy: 0.3118 - val_loss: 17.6713 - val_accuracy: 0.0857
Epoch 4/100
12/12 [==============================] - 3s 252ms/step - loss: 1.5137 - accuracy: 0.4409 - val_loss: 9.7365 - val_accuracy: 0.0857
Epoch 5/100
12/12 [==============================] - 3s 250ms/step - loss: 1.4469 - accuracy: 0.4704 - val_loss: 6.3651 - val_accuracy: 0.0857
Epoch 6/100
12/12 [==============================] - 3s 253ms/step - loss: 1.0202 - accuracy: 0.6183 - val_loss: 5.3543 - val_accuracy: 0.0857
Epoch 7/100
12/12 [==============================] - 3s 254ms/step - loss: 0.8525 - accuracy: 0.7043 - val_loss: 3.5473 - val_accuracy: 0.

In [18]:
# Define the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(timesteps, features_per_timestep, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_categorical, validation_data=(X_dev, y_dev_categorical), epochs=10, batch_size=32)

Epoch 1/10
12/12 [==============================] - 3s 193ms/step - loss: 2.4991 - accuracy: 0.1237 - val_loss: 2.1599 - val_accuracy: 0.1714
Epoch 2/10
12/12 [==============================] - 2s 182ms/step - loss: 2.1035 - accuracy: 0.2258 - val_loss: 2.2009 - val_accuracy: 0.1143
Epoch 3/10
12/12 [==============================] - 2s 182ms/step - loss: 1.9711 - accuracy: 0.2930 - val_loss: 2.2774 - val_accuracy: 0.0571
Epoch 4/10
12/12 [==============================] - 2s 183ms/step - loss: 1.7737 - accuracy: 0.3495 - val_loss: 2.2009 - val_accuracy: 0.0571
Epoch 5/10
12/12 [==============================] - 2s 201ms/step - loss: 1.5500 - accuracy: 0.4435 - val_loss: 2.3539 - val_accuracy: 0.0571
Epoch 6/10
12/12 [==============================] - 2s 180ms/step - loss: 1.3487 - accuracy: 0.4919 - val_loss: 2.3491 - val_accuracy: 0.0571
Epoch 7/10
12/12 [==============================] - 2s 181ms/step - loss: 1.1302 - accuracy: 0.5753 - val_loss: 2.5459 - val_accuracy: 0.0857
Epoch 

In [19]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.fit(X_train, y_train_categorical, validation_data=(X_dev, y_dev_categorical), epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
12/12 [==============================] - 2s 182ms/step - loss: 0.7003 - accuracy: 0.7473 - val_loss: 3.2893 - val_accuracy: 0.0857
Epoch 2/100
12/12 [==============================] - 2s 181ms/step - loss: 0.6686 - accuracy: 0.7312 - val_loss: 3.5099 - val_accuracy: 0.1143
Epoch 3/100
12/12 [==============================] - 2s 179ms/step - loss: 0.5886 - accuracy: 0.8091 - val_loss: 3.8962 - val_accuracy: 0.0857
Epoch 4/100
12/12 [==============================] - 2s 196ms/step - loss: 0.5745 - accuracy: 0.7796 - val_loss: 4.0184 - val_accuracy: 0.1429
Epoch 5/100
12/12 [==============================] - 2s 182ms/step - loss: 0.5414 - accuracy: 0.7823 - val_loss: 4.0146 - val_accuracy: 0.1143
Epoch 6/100
12/12 [==============================] - 2s 179ms/step - loss: 0.4644 - accuracy: 0.8387 - val_loss: 4.8564 - val_accuracy: 0.1143


In [20]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('best_weights.h5', save_best_only=True, monitor='val_loss', mode='min')

In [21]:
# This callback will monitor the validation loss (val_loss) of the model.
# If the validation loss doesn't improve for 5 (patience=5) consecutive epochs, it'll stop the training.
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# fitting the model, include the callback in the 'callbacks' parameter.
model.fit(X_train, y_train_categorical, validation_data=(X_dev, y_dev_categorical), epochs=100, batch_size=32, callbacks=[early_stopping])

early_stopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
model.fit(X_train, y_train_categorical, validation_data=(X_dev, y_dev_categorical), epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
12/12 [==============================] - 2s 182ms/step - loss: 0.4240 - accuracy: 0.8441 - val_loss: 5.0310 - val_accuracy: 0.1714
Epoch 2/100
12/12 [==============================] - 2s 181ms/step - loss: 0.4258 - accuracy: 0.8387 - val_loss: 5.4039 - val_accuracy: 0.0857
Epoch 3/100
12/12 [==============================] - 2s 197ms/step - loss: 0.5043 - accuracy: 0.8226 - val_loss: 5.7253 - val_accuracy: 0.1143
Epoch 4/100
12/12 [==============================] - 2s 181ms/step - loss: 0.4866 - accuracy: 0.8118 - val_loss: 5.9873 - val_accuracy: 0.0857
Epoch 5/100
12/12 [==============================] - 2s 180ms/step - loss: 0.4122 - accuracy: 0.8522 - val_loss: 6.1570 - val_accuracy: 0.1714
Epoch 6/100
12/12 [==============================] - 2s 181ms/step - loss: 0.3591 - accuracy: 0.9005 - val_loss: 6.3307 - val_accuracy: 0.0857
Epoch 1/100
12/12 [==============================] - 2s 185ms/step - loss: 0.3314 - accuracy: 0.8602 - val_loss: 6.4573 - val_accuracy: 0.1143

In [22]:
from keras.regularizers import l2

model = Sequential()

# First Convolutional Layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))  # Add dropout of 25%

# Second Convolutional Layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))  # Add dropout of 25%

# Flatten and Fully Connected Layers
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))  # L2 regularization
model.add(Dropout(0.5))  # Add dropout of 50%
model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train_categorical, validation_data=(X_dev, y_dev_categorical), epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
12/12 [==============================] - 5s 299ms/step - loss: 6.7892 - accuracy: 0.1398 - val_loss: 4.1377 - val_accuracy: 0.1143
Epoch 2/100
12/12 [==============================] - 3s 286ms/step - loss: 3.5365 - accuracy: 0.1237 - val_loss: 3.2549 - val_accuracy: 0.0857
Epoch 3/100
12/12 [==============================] - 4s 313ms/step - loss: 2.9314 - accuracy: 0.1156 - val_loss: 2.6809 - val_accuracy: 0.1429
Epoch 4/100
12/12 [==============================] - 4s 292ms/step - loss: 2.5522 - accuracy: 0.1183 - val_loss: 2.4340 - val_accuracy: 0.1143
Epoch 5/100
12/12 [==============================] - 3s 287ms/step - loss: 2.4107 - accuracy: 0.1075 - val_loss: 2.3283 - val_accuracy: 0.1429
Epoch 6/100
12/12 [==============================] - 4s 304ms/step - loss: 2.3327 - accuracy: 0.1183 - val_loss: 2.2843 - val_accuracy: 0.1714
Epoch 7/100
12/12 [==============================] - 3s 291ms/step - loss: 2.2906 - accuracy: 0.1237 - val_loss: 2.2441 - val_accuracy: 0.1714

In [23]:
model = Sequential()

# First Convolutional Layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second Convolutional Layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and Fully Connected Layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

# Use RMSprop optimizer as an alternative
optimizer = RMSprop(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train_categorical, validation_data=(X_dev, y_dev_categorical), epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
12/12 [==============================] - 4s 273ms/step - loss: 13.6921 - accuracy: 0.1425 - val_loss: 4.0276 - val_accuracy: 0.0857
Epoch 2/100
12/12 [==============================] - 3s 251ms/step - loss: 2.3536 - accuracy: 0.2097 - val_loss: 3.9284 - val_accuracy: 0.1143
Epoch 3/100
12/12 [==============================] - 3s 262ms/step - loss: 1.9817 - accuracy: 0.3118 - val_loss: 3.1915 - val_accuracy: 0.1143
Epoch 4/100
12/12 [==============================] - 3s 275ms/step - loss: 1.8071 - accuracy: 0.3817 - val_loss: 2.4970 - val_accuracy: 0.0571
Epoch 5/100
12/12 [==============================] - 3s 257ms/step - loss: 1.5884 - accuracy: 0.4570 - val_loss: 2.2640 - val_accuracy: 0.1143
Epoch 6/100
12/12 [==============================] - 3s 258ms/step - loss: 1.5649 - accuracy: 0.4355 - val_loss: 2.2647 - val_accuracy: 0.1143
Epoch 7/100
12/12 [==============================] - 3s 259ms/step - loss: 1.3172 - accuracy: 0.5269 - val_loss: 2.5050 - val_accuracy: 0.114

In [25]:
import tensorflow as tf
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

# Sample data
X = np.random.rand(1000, 50)  # Example feature data of shape (1000 samples, 50 features)
y = np.random.choice(['mozart', 'chopin', 'handel', 'byrd', 'schumann', 'mendelssohn', 'hummel', 'bach', 'bartok'], 1000)  # Example labels

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Calculate the number of unique classes
num_classes = len(np.unique(y_encoded))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Fine-tuning (with a reduced learning rate, for example)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_fine_tune = model.fit(X_train, y_train, epochs=5, validation_split=0.2)

# Prediction
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Evaluation
print(classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_))


Epoch 1/10
20/20 [==============================] - 0s 7ms/step - loss: 2.2785 - accuracy: 0.1219 - val_loss: 2.2299 - val_accuracy: 0.1187
Epoch 2/10
20/20 [==============================] - 0s 2ms/step - loss: 2.2366 - accuracy: 0.1312 - val_loss: 2.2347 - val_accuracy: 0.1312
Epoch 3/10
20/20 [==============================] - 0s 2ms/step - loss: 2.2209 - accuracy: 0.1406 - val_loss: 2.2305 - val_accuracy: 0.1063
Epoch 4/10
20/20 [==============================] - 0s 3ms/step - loss: 2.2001 - accuracy: 0.1422 - val_loss: 2.2221 - val_accuracy: 0.1125
Epoch 5/10
20/20 [==============================] - 0s 2ms/step - loss: 2.1806 - accuracy: 0.1656 - val_loss: 2.2265 - val_accuracy: 0.1063
Epoch 6/10
20/20 [==============================] - 0s 2ms/step - loss: 2.1739 - accuracy: 0.1500 - val_loss: 2.2238 - val_accuracy: 0.1063
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 2.1443 - accuracy: 0.1828 - val_loss: 2.2224 - val_accuracy: 0.1125
Epoch 8/10
20/20 [==

/Users/fedijay/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fedijay/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fedijay/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
num_classes = len(np.unique(y_encoded))
print(num_classes)

9


In [27]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2)

# Define the model - Adjust input_shape and number of dense units as per dataset's requirement
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Fine-tuning
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_fine_tune = model.fit(X_train, y_train, epochs=5, validation_split=0.2)

# Prediction
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Evaluation
f1 = f1_score(y_test, y_pred_classes, average=None)
precision = precision_score(y_test, y_pred_classes, average=None)
recall = recall_score(y_test, y_pred_classes, average=None)

print("Class-wise metrics:")
print("Class\t\tPrecision\tRecall\t\tF1")
for i, composer in enumerate(label_encoder.classes_):
    print(f"{composer}\t{precision[i]:.2f}\t\t{recall[i]:.2f}\t\t{f1[i]:.2f}")

print("\nOverall Metrics:")
print(classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_))

Epoch 1/10
20/20 [==============================] - 0s 7ms/step - loss: 2.2944 - accuracy: 0.0922 - val_loss: 2.2021 - val_accuracy: 0.1250
Epoch 2/10
20/20 [==============================] - 0s 2ms/step - loss: 2.2318 - accuracy: 0.1172 - val_loss: 2.1943 - val_accuracy: 0.1562
Epoch 3/10
20/20 [==============================] - 0s 2ms/step - loss: 2.2013 - accuracy: 0.1437 - val_loss: 2.2065 - val_accuracy: 0.1125
Epoch 4/10
20/20 [==============================] - 0s 2ms/step - loss: 2.1844 - accuracy: 0.1578 - val_loss: 2.2056 - val_accuracy: 0.1562
Epoch 5/10
20/20 [==============================] - 0s 2ms/step - loss: 2.1871 - accuracy: 0.1547 - val_loss: 2.2016 - val_accuracy: 0.1187
Epoch 6/10
20/20 [==============================] - 0s 2ms/step - loss: 2.1542 - accuracy: 0.1609 - val_loss: 2.2035 - val_accuracy: 0.1125
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 2.1550 - accuracy: 0.1734 - val_loss: 2.2015 - val_accuracy: 0.1250
Epoch 8/10
20/20 [==